In [ ]:
!export JVM_ARGS="-Xmx2048m -XX:MaxPermSize=1024m"

In [ ]:
!echo $JVM_ARGS

In [ ]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder 
    .master("local[*]") 
    .config("spark.driver.memory", "15g") 
    .appName("my-assignment-7") 
    .getOrCreate()

In [ ]:
spark.conf.getAll

In [ ]:
//Spark SQL Home Assignment Problem Solution
import org.apache.spark.sql.{DataFrame, SQLContext, SparkSession}

def GetJsonWebData(url: String): DataFrame ={
    //Grab the contents of the web page at url
    val rdd = spark.sparkContext.parallelize(scala.io.Source.fromURL(url).mkString :: Nil) 
    //Convert it into JSON (must be single line JSON)
    val df = spark.read.json(rdd)
    return df
}  

// CODE FOR SETTING UP THE URL AND GETTING THE DATA INTO A DATAFRAME GOES HERE
val selection = "borough,crash_date,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed"
val url = s"https://data.cityofnewyork.us/resource/h9gi-nx95.json?$$select=$selection&$$where=number_of_persons_injured>0%20OR%20number_of_persons_killed>0&$$limit=1700000"
val response = GetJsonWebData(url)



In [9]:
response.schema

res4: org.apache.spark.sql.types.StructType = StructType(StructField(borough,StringType,true),StructField(crash_date,StringType,true),StructField(number_of_pedestrians_injured,StringType,true),StructField(number_of_pedestrians_killed,StringType,true),StructField(number_of_persons_injured,StringType,true),StructField(number_of_persons_killed,StringType,true))


In [6]:
//Spark SQL Home Assignment Problem Solution
import org.apache.spark.sql.{DataFrame, SQLContext, SparkSession}

def GetJsonWebData(url: String): DataFrame ={
    //Grab the contents of the web page at url
    val rdd = spark.sparkContext.parallelize(scala.io.Source.fromURL(url).mkString :: Nil) 
    //Convert it into JSON (must be single line JSON)
    val df = spark.read.json(rdd)
    return df
}  

// CODE FOR SETTING UP THE URL AND GETTING THE DATA INTO A DATAFRAME GOES HERE
val selection = "borough,crash_date,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed"
val url = s"https://data.cityofnewyork.us/resource/h9gi-nx95.json?$$select=$selection&$$where=number_of_persons_injured>0%20OR%20number_of_persons_killed>0&$$limit=1700000"
val response = GetJsonWebData(url)




// CODE FOR CREATING A COLUMN WITH THE YEAR GOES HERE
import org.apache.spark.sql.functions.udf
def slice_col(n: Int) =
udf(
  (value: String) => value.slice(0,n))
val df_year = response.withColumn("year", slice_col(4)($"crash_date"))



// CODE FOR CALCULATING PROPORTIONS FOR EACH YEAR GOES HERE
val df_proportions = 
    df_year.select("year","number_of_pedestrians_injured","number_of_pedestrians_killed",
               "number_of_persons_injured","number_of_persons_killed")
        .groupBy($"year").agg(sum("number_of_pedestrians_injured"),sum("number_of_pedestrians_killed"),
                                   sum("number_of_persons_injured"),sum("number_of_persons_killed"))
        .orderBy("year")
        .withColumnRenamed("sum(number_of_pedestrians_injured)","total_ped_injuries")
        .withColumnRenamed("sum(number_of_pedestrians_killed)","total_ped_killed")
        .withColumnRenamed("sum(number_of_persons_injured)","total_injuries")
        .withColumnRenamed("sum(number_of_persons_killed)","total_killed")
        .withColumn("pedestrian_proportion",col("total_ped_injuries")/col("total_ped_killed"))
        .withColumn("total_proportion",(col("total_injuries")/col("total_killed")))
        //.select("year","pedestrian_proportion","total_proportion")
        .withColumn("pedestrian_proportion",format_number($"pedestrian_proportion", 2))
        .withColumn("total_proportion",format_number($"total_proportion", 2))




// CODE FOR CALCULATING PROPORTIONS BY YEAR FOR EACH BOROUGH GOES HERE
val borough_list = List("BRONX","MANHATTAN","STATEN ISLAND","QUEENS","BROOKLYN")
val df_borough = df_year.select("borough","year","number_of_pedestrians_injured","number_of_pedestrians_killed",
               "number_of_persons_injured","number_of_persons_killed")
        .filter(col("borough").isin(borough_list:_*))
        .groupBy("borough","year").agg(sum("number_of_pedestrians_injured"),sum("number_of_pedestrians_killed"),
                                   sum("number_of_persons_injured"),sum("number_of_persons_killed"))
        .orderBy("borough","year")
        .withColumnRenamed("sum(number_of_pedestrians_injured)","total_ped_injuries")
        .withColumnRenamed("sum(number_of_pedestrians_killed)","total_ped_killed")
        .withColumnRenamed("sum(number_of_persons_injured)","total_injuries")
        .withColumnRenamed("sum(number_of_persons_killed)","total_killed")
        .withColumn("pedestrian_proportion",col("total_ped_injuries")/col("total_ped_killed"))
        .withColumn("total_proportion",col("total_injuries")/col("total_killed"))
        //.select("borough","year","pedestrian_proportion","total_proportion")
        .withColumn("pedestrian_proportion",format_number($"pedestrian_proportion", 2))
        .withColumn("total_proportion",format_number($"total_proportion", 2))
                


// EXTRACT DATA FOR EACH BOROUGH INTO A DATAFRAME
val manhattan = df_borough.filter($"borough"==="MANHATTAN")
val staten_island = df_borough.filter($"borough"==="STATEN ISLAND")
val queens = df_borough.filter($"borough"==="QUEENS")
val bronx = df_borough.filter($"borough"==="BRONX")
val brooklyn = df_borough.filter($"borough"==="BROOKLYN")

//import org.apache.spark.sql.functions._
//res.withColumn("c3", format_number(col("c1")/col("c2"), 4)).orderBy(col("c3").desc).show()

//CODE FOR SHOWING ALL THE RESULTS GOES HERE


//df_proportions.show //shows proportions for each year

//For each year by borough

//staten_island.show
//queens.show
//bronx.show
//brooklyn.show


import org.apache.spark.sql.{DataFrame, SQLContext, SparkSession}
GetJsonWebData: (url: String)org.apache.spark.sql.DataFrame
selection: String = borough,crash_date,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed
url: String = https://data.cityofnewyork.us/resource/h9gi-nx95.json?$select=borough,crash_date,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed&$where=number_of_persons_injured>0%20OR%20number_of_persons_killed>0&$limit=1700000
response: org.apache.spark.sql.DataFrame = [borough: string, crash_date: string ... 4 more fields]
import org.apache.spark.sql.functions.udf
slice_col: (n: Int)org.apache.spark.sql.expressions.UserDefinedFunction
df_year: org.apache....


In [7]:
df_proportions.show

+----+------------------+----------------+--------------+------------+---------------------+----------------+
|year|total_ped_injuries|total_ped_killed|total_injuries|total_killed|pedestrian_proportion|total_proportion|
+----+------------------+----------------+--------------+------------+---------------------+----------------+
|2012|            5906.0|            72.0|       27453.0|       137.0|                82.03|          200.39|
|2013|           11988.0|           176.0|       55128.0|       297.0|                68.11|          185.62|
|2014|           11036.0|           133.0|       51223.0|       262.0|                82.98|          195.51|
|2015|           10084.0|           133.0|       51358.0|       243.0|                75.82|          211.35|
|2016|           11089.0|           149.0|       60317.0|       246.0|                74.42|          245.19|
|2017|           11149.0|           127.0|       60656.0|       256.0|                87.79|          236.94|
|2018|    

In [2]:
df_proportions.select("year","pedestrian_proportion","total_proportion").show
manhattan.select("borough","year","pedestrian_proportion","total_proportion").show
staten_island.select("borough","year","pedestrian_proportion","total_proportion").show
queens.select("borough","year","pedestrian_proportion","total_proportion").show
bronx.select("borough","year","pedestrian_proportion","total_proportion").show
brooklyn.select("borough","year","pedestrian_proportion","total_proportion").show

+----+---------------------+----------------+
|year|pedestrian_proportion|total_proportion|
+----+---------------------+----------------+
|2012|                82.03|          200.39|
|2013|                68.11|          185.62|
|2014|                82.98|          195.51|
|2015|                75.82|          211.35|
|2016|                74.42|          245.19|
|2017|                87.79|          236.94|
|2018|                90.41|          268.14|
|2019|                80.67|          251.59|
|2020|                66.25|          165.86|
|2021|                57.26|          174.93|
|2022|                67.99|          180.93|
|2023|                96.96|          212.65|
+----+---------------------+----------------+

+---------+----+---------------------+----------------+
|  borough|year|pedestrian_proportion|total_proportion|
+---------+----+---------------------+----------------+
|MANHATTAN|2012|                81.76|          184.19|
|MANHATTAN|2013|                96.93| 

<h1>PART 1</h1>
When NYC Mayor Bill D'Blasio assumed office in January 2014, one of his first signature initiatives was making the city safer for pedestrians and motorists. Amongst the many initiatives to make the streets safer was the institution of a low 25 m.p.h. speed limit within the city limits. The reasoning was that car crashes at lower speeds lead to fewer fatalities than crashes at higher speeds.

The goal of this assignment is to test whether this has worked or not. Well, actually, the goal is to practice some Spark SQL but we'll do a simple test along the way!

We'll use the Motor Vehicle Collision data available from NYC Open Data (the url is below). You need to do the following:

<ol>
    <li>Use the Socrata API to get Motor Vehicle Collision data from NYC open data (links to the relevant documentation are in the Web resources cell below)</li>
    <li>Your API call should return only the following data items:</li>
    <ol>
        <li>borough</li>
        <li>crash date</li>
        <li>number of pedestrians injured</li>
        <li>number of pedestrians killed</li>
        <li>number of persons injured</li>
        <li>number of persons killed</li>
    </ol>
    <li>Additionally, get only the data for which the number of persons injured or the number of persons killed is greater than zero</li>
    <li>Once you've created the url for the API call, use the GetJsonWebData function (defined below) to get the data into a dataframe. Note that, in a url, spaces should be replaced by %20</li>
    <li>Next, replace the crash_date column with the year of the crash. The scala slice function will help you here and you may need to define a udf. Call this new column <span style="color:blue">year</span></li>
    <li>Group the data by year and report the following statistics:</li>
    <ol>
        <li>proportion of pedestrian injuries to pedestrian deaths</li>
        <li>proportion of people injured to people deaths</li>
        <li>roughly, if the speed limits worked, then both these proportions should be going up</li>
    </ol>
    <li>Make sure that the results are ordered by year!</li>
    <li>Make sure that floating point numbers are formatted to two decimal places (see formatting at the bottom of this notebook)</li>
    <li>Show the result (e.g., below)</li>
    
</ol>

If you run into Java memory issues, try using the java buffer command below. You can also try restarting the kernel and seeing if that fixes it. If all else fails, move to a cluster on the cloud!


    


<h3>Format of the output</h3>
<pre>    
       
+----+---------------------+----------------+
|year|pedestrian_proportion|total_proportion|
+----+---------------------+----------------+
|2012|                82.03|          200.39|
|2013|                68.11|          185.62|
|2014|                82.98|          195.51|
|2015|                75.82|          211.35|
|2016|                74.42|          245.19|
|2017|                87.79|          236.94|
|2018|                90.41|          268.14|
|2019|                80.67|          251.59|
|2020|                66.25|          165.85|
|2021|                57.26|          174.93|
|2022|                67.99|          180.93|
|2023|                96.81|          211.17|
+----+---------------------+----------------+

</pre>

<h1>PART 2</h1>
Next we'll do the same thing we did in Part 1 except that we'll see if there are differences by borough in how well the speed limit reduction has worked. 

Do the following:

<ol>
    <li>Create a scala list that contains the names of the boroughs. We will clean our data to eliminate any entries that don't have a proper borough name</li>
    <li>Remove all data that has an entry in the borough name that doesn't correspond to a name in our list (see the filter example below on how to do this)</li>
    <li>Now compute the same proportions as in part 1 but on data that is grouped by first the borough and then the year (you can use <span style="color:blue">groupBy("borough","year")</span>)</li>
    <li>Create dataframes for the results for each borough</li>
    <li>Show all the results</li>
    
</ol>
<h3>Sample output</h3>
<pre>
+---------+----+---------------------+----------------+
|  borough|year|pedestrian_proportion|total_proportion|
+---------+----+---------------------+----------------+
|MANHATTAN|2012|                81.76|          184.19|
|MANHATTAN|2013|                96.93|          194.49|
|MANHATTAN|2014|               103.44|          197.40|
|MANHATTAN|2015|               127.11|          296.86|
|MANHATTAN|2016|                69.17|          183.41|
|MANHATTAN|2017|                84.24|          171.15|
|MANHATTAN|2018|               137.08|          271.80|
|MANHATTAN|2019|                78.25|          218.42|
|MANHATTAN|2020|                98.00|          170.33|
|MANHATTAN|2021|                65.88|          178.08|
|MANHATTAN|2022|                75.50|          213.79|
|MANHATTAN|2023|               126.38|          236.56|
+---------+----+---------------------+----------------+

+-------------+----+---------------------+----------------+
|      borough|year|pedestrian_proportion|total_proportion|
+-------------+----+---------------------+----------------+
|STATEN ISLAND|2012|                67.00|          113.18|
|STATEN ISLAND|2013|               105.33|          527.75|
|STATEN ISLAND|2014|                46.00|          157.70|
|STATEN ISLAND|2015|                47.20|          129.00|
|STATEN ISLAND|2016|                77.33|          164.89|
|STATEN ISLAND|2017|                60.00|          193.75|
|STATEN ISLAND|2018|                60.00|          191.43|
|STATEN ISLAND|2019|               109.50|          350.75|
|STATEN ISLAND|2020|                41.33|          116.88|
|STATEN ISLAND|2021|                57.50|          182.50|
|STATEN ISLAND|2022|                70.00|          153.38|
|STATEN ISLAND|2023|               153.00|          332.67|
+-------------+----+---------------------+----------------+

+-------+----+---------------------+----------------+
|borough|year|pedestrian_proportion|total_proportion|
+-------+----+---------------------+----------------+
| QUEENS|2012|                78.13|          196.64|
| QUEENS|2013|                51.71|          150.54|
| QUEENS|2014|                84.31|          192.90|
| QUEENS|2015|                80.16|          204.37|
| QUEENS|2016|               109.47|          352.52|
| QUEENS|2017|                83.61|          252.57|
| QUEENS|2018|                65.03|          204.11|
| QUEENS|2019|                78.23|          270.61|
| QUEENS|2020|                64.35|          158.00|
| QUEENS|2021|                82.25|          248.53|
| QUEENS|2022|               119.64|          274.71|
| QUEENS|2023|               137.22|          229.52|
+-------+----+---------------------+----------------+

+-------+----+---------------------+----------------+
|borough|year|pedestrian_proportion|total_proportion|
+-------+----+---------------------+----------------+
|  BRONX|2012|                94.12|          169.29|
|  BRONX|2013|                66.88|          195.59|
|  BRONX|2014|               150.50|          309.79|
|  BRONX|2015|               124.73|          216.38|
|  BRONX|2016|                96.07|          318.42|
|  BRONX|2017|               135.40|          343.76|
|  BRONX|2018|               144.40|          299.48|
|  BRONX|2019|               156.67|          593.45|
|  BRONX|2020|                98.70|          226.87|
|  BRONX|2021|               226.80|          250.30|
|  BRONX|2022|                99.33|          176.58|
|  BRONX|2023|               161.60|          276.00|
+-------+----+---------------------+----------------+

+--------+----+---------------------+----------------+
| borough|year|pedestrian_proportion|total_proportion|
+--------+----+---------------------+----------------+
|BROOKLYN|2012|                96.63|          228.06|
|BROOKLYN|2013|               121.37|          230.04|
|BROOKLYN|2014|                79.71|          212.61|
|BROOKLYN|2015|                65.83|          204.57|
|BROOKLYN|2016|               145.85|          363.57|
|BROOKLYN|2017|               133.15|          301.18|
|BROOKLYN|2018|               121.09|          338.41|
|BROOKLYN|2019|                81.82|          220.47|
|BROOKLYN|2020|               104.71|          190.18|
|BROOKLYN|2021|                59.53|          216.20|
|BROOKLYN|2022|               103.45|          244.96|
|BROOKLYN|2023|               147.33|          270.69|
+--------+----+---------------------+----------------+

</pre>

<h4>And, similarly for the other boroughs</h4>


<h1>Web resources</h1>
<li><a href="https://data.cityofnewyork.us/resource/h9gi-nx95.json">The data URL</a></li>
<li><a href="https://dev.socrata.com/docs/queries/where.html">Socrata Where clause</a></li>
<li><a href="https://dev.socrata.com/docs/queries/select.html">Socrata select clause</a></li>
<li><a href="https://dev.socrata.com/docs/queries/limit.html">Socrata limit clause</a></li>
<li><a href="https://learning.oreilly.com/library/view/scala-cookbook/9781449340292/ch10s19.html">Scala slice</a></li>

<h1>Grab JSON from a URL function</h1>

In [8]:
import org.apache.spark.sql.{DataFrame, SQLContext, SparkSession}

def GetJsonWebData(url: String): DataFrame ={
    //Grab the contents of the web page at url
    val rdd = spark.sparkContext.parallelize(scala.io.Source.fromURL(url).mkString :: Nil) 
    //Convert it into JSON (must be single line JSON)
    val df = spark.read.json(rdd)
    return df
}  

import org.apache.spark.sql.{DataFrame, SQLContext, SparkSession}
GetJsonWebData: (url: String)org.apache.spark.sql.DataFrame


<h1>If (when?) you run out of memory</h1>
<li>Create a spark session with enough local memory</li>
<li>Do this at the top of the notebook after restarting the kernel!</li>
<li>Or try using the cluster</li>
<li>If you run out of memory on the cluster</li>
<ul><li>restart the cluster kernel and add the following line in the topmost cell (and run it)</li>
!export JVM_ARGS="-Xmx2048m -XX:MaxPermSize=1024m"

In [ ]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder 
    .master("local[*]") 
    .config("spark.driver.memory", "15g") 
    .appName("my-assignment-7") 
    .getOrCreate()

In [ ]:
!export JVM_ARGS="-Xmx2048m -XX:MaxPermSize=1024m"


<h1>filter (example)</h1>

In [ ]:
val df = Seq(("John","NYC",1000),
             ("JILL","BOSTON",1700),
            ("JAMES","NYC",800),
            ("JACQUES","BOSTON",400))
        .toDF("Name","City","Score")

//Note the === (3 = signs) for matching to string values in the example below
df.groupBy("City").sum("score").filter($"City"==="BOSTON").show

<h1>Formatting decimal numbers</h1>

In [ ]:
val df = Seq(("John",23.23435353),
             ("Jill",453.23233),
             ("Qing",3435.4353524),
             ("Angeliki",0.24353545453)).toDF("name","score")
df.show

In [ ]:
val new_df = df.withColumn("score",format_number($"score", 2))
new_df.show

In [ ]:
df_year.select("borough").distinct.show